In [1]:
#Import the necessary packages
import requests
from requests_html import AsyncHTMLSession
from bs4 import BeautifulSoup
import pandas as pd
import warnings 
from details import agt, cook

warnings.filterwarnings('ignore')

In [2]:
#Create the base url  
base_url = 'https://www.realtytrac.com/'

#Create a search for the LA area
la_search = 'mapsearch/sold/ca/los-angeles-county/'

In [3]:
#Grab the website content and store it in a BeautifulSoup object
response = requests.get(base_url+la_search, headers=agt, cookies=cook)
soup = BeautifulSoup(response.content, 'html.parser')

In [4]:
houses = soup.find_all('div', class_="house alt clearfix")

#Number of pages to run the scraper through
num_pages = 300

print(f'You\'ll scrape {len(houses)*num_pages} houses')

You'll scrape 3000 houses


In [15]:
absolute_links = []

for i in range(num_pages):
    page = soup.find_all('div', class_="house alt clearfix")
    
    for house in page:
        absolute_links.append(base_url+house.find('a')['href'])
    
    next_link = soup.find('a', class_='next')['href']
    response = requests.get(base_url+next_link, headers=agt, cookies=cook)
    soup = BeautifulSoup(response.content, 'html.parser')
    
len(absolute_links)

3000

In [5]:
#Create the function that extracts the attributes for each house(link)

#details_df = pd.DataFrame()

def find_details(house):
    
    
    try:
        lat = house.find('meta', property='og:latitude')['content']
    except:
        lat=None
    try:
        long = house.find('meta', property='og:longitude')['content'] 
    except:
        long=None
    try:    
        address = house.find('span', class_='address heading').text 
    except:
        address=None
    
    #Property Details
    try:
        details = house.find('ul', class_='detail-list')
    except:
        details = None
    try:
        add_details = details.find_all('li')
    except:
        add_details = None
    
    try:
        prop_type = add_details[0].find('span', itemprop='name').text
    except:
        prop_type=None
    try:
        rooms = add_details[1].find('span', itemprop='description').text
    except:
        rooms=None
    try:
        home_size_p = details.find('span', itemprop='name', text='Home Size').parent
        home_size = home_size_p.find('span', itemprop='value').text
    except:
        home_size=''
    try:                                                              
        home_unit = home_size_p.find('span', itemprop='unitText').text
    except:
        home_unit=''
    try:
        lot_size_p = details.find('span', itemprop='name', text='Lot Size').parent
        lot_size = lot_size_p.find('span', itemprop='value').text
    except:
        lot_size=''
    try:                                                             
        lot_unit = lot_size_p.find('span', itemprop='unitText').text
    except:
        lot_unit=''
    try:
        year_built_p = details.find('span', itemprop='description', text='Built in').parent
        year_built = year_built_p.find('span', itemprop='value').text
    except:
        year_built=None
    try:
        parcel_p = details.find('span', itemprop='name', text='Parcel Number').parent
        parcel = parcel_p.find('span', itemprop='propertyID').text
    except:
        parcel=None
    try:
        realty_p = details.find('span', itemprop='name', text='RealtyTrac Property ID').parent
        realty = realty_p.find('span', itemprop='value').text
    except:
        realty=None
    try:
        county_p = details.find('span', itemprop='name', text='County').parent
        county = county_p.find('span', itemprop='description').text
    except:
        county=None
    try:
        sub_p = details.find('span', itemprop='name', text='Subdivision').parent
        sub = sub_p.find('span', itemprop='value').text
    except:
        sub=None
    try:
        census_p = details.find('span', itemprop='name', text='Census').parent
        census = census_p.find('span', itemprop='value').text
    except:
        census=None
    try:
        tract_p = details.find('span', itemprop='name', text='Tract').parent
        tract = tract_p.find('span', itemprop='value').text
    except:
        tract=None
    try:
        zoning = details.find_all('li')[-2].text
    except:
        zoning=None
    try:                                                             
        lot_p = details.find('span', itemprop='name', text='Lot').parent
        lot = lot_p.find('span', itemprop='value').text
    except:
        lot = None
    
    
    #Sales History
    try:
        sales = house.find('section', id='occupancy-link') 
    except:
        sales = None
    
    try:                                                              
        sale_price = sales.find('label', attrs={'for':'PurchasePrice'}).parent.parent.find('td', class_='col2').text            
    except:
        sale_price=None
    try:
        date = sales.find('label', attrs={'for':'PurchaseDate'}).parent.parent.find('td', class_='col2').text
    except:
        date=None
    
    #Estimated Value
    try:
        est = house.find('div', class_='rate-row access')
    except:
        est = None
    
    try:                                                              
        estimate = est.find('strong', class_='price').text
    except:
        estimate= None
    
    #Home Disclosures
    try:
        disc = house.find('div', id="collapsePropertyHomeDisclosuresArea")
    except:
        disc = None
    
    try:
        sex_offenders = disc.find('span', class_='hd-title', text='Sex Offenders').parent.find('span', class_='hd-label').text
    except:
        sex_offenders = None
    
    try:
        crime_index = disc.find('span', class_='hd-title',text='Crime Index').parent.find('span', class_='hd-label').text
    except:
        crime_index = None
    
    try:
        env_hazards = disc.find('span', class_='hd-title',text='Environmental Hazards').parent.find('span', class_='hd-label').text
    except:
        env_hazards = None
    
    try:
        natural_disasters = disc.find('span', class_='hd-title', text = 'Natural Disasters').parent.find('span', class_='hd-label').text
    except:
        natural_disasters = None
    try:
        school_index = disc.find('span', class_='hd-title', text = 'Local Schools').parent.find('span', class_='hd-label').text
    except:
        school_index = None
    
    try:
        url = house.find('meta', property='og:url')['content']
    except:
        url=None
    #Create the df to be returned
    details_df = pd.DataFrame({
        
                       'latitude': [lat],
                       'longitude': [long],
                       'address':[address],
                       'property_type': [prop_type],
                       'rooms': [rooms],
                       'home_size': [home_size+home_unit],
                       'lot_size': [lot_size+lot_unit],
                       'year_built': [year_built],
                       'parcel_number': [parcel],
                       'realtyID': [realty],
                       'county': [county],
                       'subdivision': [sub],
                       'census': [census],
                       'tract': [tract],
                       'lot': [lot],
                       'zoning': [zoning],
                       'date': [date],
                       'sale_price': [sale_price],
                       'estimated_value': [estimate],
                       'sex_offenders': [sex_offenders],
                       'crime_index': [crime_index],
                       'enviornmental_hazards': [env_hazards],
                       'natural_disasters': [natural_disasters],
                       'school_quality': [school_index],
                       'url': [url]
                      })
    
    return details_df

In [6]:
#Create a function that captures sales history data

def find_sales_history(house):
    
    temp = pd.DataFrame()
    
    try:
        details = house.find('ul', class_='detail-list')
    except:
        details = None
   
    try:
        parcel_p = details.find('span', itemprop='name', text='Parcel Number').parent
        parcel = parcel_p.find('span', itemprop='propertyID').text
    except:
        parcel=None
    
    try:
        table = house.find('table', class_='sales-table history-table').find('tbody')
    except:
        table = None
    
    try:
        sales = table.find_all('tr')
    except:
        sales = None
    
    if sales !=None:
        
        for i in range(len(sales)):
            
            try:
                date = sales[i].find('td', class_='col-1').text
            except:
                date = None
        
            try:
                price = sales[i].find('td', class_='col-3').text
            except:
                price = None
            
            try:
                price_sqft = sales[i].find('td', class_='col-2').text
            except:
                price_sqft = None
            
        
            details_df = pd.DataFrame({
                
                    'parcel_number': [parcel],
                    'date': [date],
                    'price': [price],
                    'price_sqft': [price_sqft]
                   
                      })
            
            temp = temp.append(details_df, ignore_index=True)
            
      
    return temp

In [12]:
#Spot check 
#df = pd.DataFrame()
#asession = AsyncHTMLSession()
#r = await asession.get(absolute_links[16])
#await r.html.arender()
#response = r.html.raw_html
#soup = BeautifulSoup(response, 'html.parser')
#df = df.append(find_sales_history(soup), ignore_index=True)
#df

In [18]:
with open("links.txt", "w") as f:
    for s in absolute_links:
        f.write(str(s) +"\n")

In [8]:
#Opening up the links file and saving it as a list
absolute_links = []

with open("links.txt", "r") as f:
    for line in f:
        absolute_links.append(line.strip())
        
len(absolute_links)

3000

In [ ]:
#Loop through all the houses captured in absolute_links
details = pd.DataFrame()
sales_history  = pd.DataFrame()
asession = AsyncHTMLSession()


for i in range(len(absolute_links[1125:])):
    r = await asession.get(absolute_links[i+1125])
    await r.html.arender()
    resp= r.html.raw_html
    soup = BeautifulSoup(resp, 'html.parser')
    details = details.append(find_details(soup), ignore_index=True)
    sales_history = sales_history.append(find_sales_history(soup), ignore_index=True)

    if (i+1)%25 == 0:
        details.to_csv('Data/house_data_details_scraped4.csv', index=False)
        sales_history.to_csv('Data/house_data_saleshistory_scraped4.csv', index=False)
        print(f'Scraped {details.shape[0]} properties')
        asession.close()
        asession = AsyncHTMLSession()

In [10]:
details.to_csv('Data/house_data_details_scraped4.csv', index=False)
sales_history.to_csv('Data/house_data_saleshistory_scraped4.csv', index=False)
details.head()

latitude    longitude               address            property_type  \
0  34.685165  -118.185494      3046 W Avenue J4  Single Family Residence   
1  34.079363  -118.277093  664 Silver Lake Blvd  Single Family Residence   
2  34.073158  -117.896497       131 N Meadow Rd  Single Family Residence   
3  33.960752  -118.350677      333 E Nutwood St          Office Building   
4  34.119065  -118.083806       6568 N Vista St  Single Family Residence   

             rooms   home_size    lot_size year_built parcel_number  \
0  4 beds, 3 baths  2,458 sqft  6,618 sqft       1988    3153044014   
1    1 bed, 1 bath    523 sqft  5,768 sqft       1921    5401022001   
2  3 beds, 2 baths  1,163 sqft  8,523 sqft       1954    8453023007   
3             None  3,985 sqft  9,078 sqft       1940    4021013008   
4  3 beds, 2 baths  2,228 sqft  7,066 sqft       1994    5381011004   

     realtyID  ...                        zoning        date  sale_price  \
0  1113047699  ...  \nZoning:\n\nLRR17500*\n\t\t  11/18/2020    $412,000   
1  1113047718  ...      \nZoning:\n\nLARD2\n\t\t  10/28/1993        None   
2  1113048513  ...  \nZoning:\n\nWCR17500*\n\t\t  11/18/2020    $625,000   
3  1113069596  ...      \nZoning:\n\nINC1*\n\t\t  11/18/2020  $1,600,000   
4  1113071022  ...     \nZoning:\n\nLCR1YY\n\t\t  11/16/1979        None   

  estimated_value sex_offenders      crime_index enviornmental_hazards  \
0        $400,500       2 Found         Moderate               2 Found   
1        $441,900      10 Found    Slightly High               5 Found   
2        $572,300       6 Found              Low               2 Found   
3            None       8 Found  Moderately High              12 Found   
4        $814,900       1 Found              Low               2 Found   

  natural_disasters school_quality  \
0           2 Found        Average   
1           1 Found        Average   
2           2 Found  Above Average   
3           1 Found           Poor   
4           1 Found  Above Average   

                                                 url  
0  https://www.realtytrac.com/property/ca/lancast...  
1  https://www.realtytrac.com/property/ca/los-ang...  
2  https://www.realtytrac.com/property/ca/west-co...  
3  https://www.realtytrac.com/property/ca/inglewo...  
4  https://www.realtytrac.com/property/ca/san-gab...  

[5 rows x 25 columns]

In [11]:
#Close out the session
asession.close()

<coroutine object AsyncHTMLSession.close at 0x11aee7e60>

In [13]:
#Save the dataframe to a csv file
#details.to_csv('Data/house_data_details_scraped.csv', index=False)
#sales_history.to_csv('Data/house_data_saleshistory_scraped.csv', index=False)